In [1]:
import numpy as np
import cv2

def seam_carve(img, scale):
    # 计算要删除的像素数
    if scale > 1:
        raise ValueError("Scale must be less than 1")
    elif scale == 1:
        return img
    
    height, width = img.shape[:2]
    new_width = int(scale * width)
    num_pixels = width - new_width
    
    # 转换为灰度图像并计算能量图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    energy_map = cv2.Canny(gray, 100, 200)
    
    for i in range(num_pixels):
        # 计算每个像素点的能量值
        M = energy_map.astype(np.float64)
        for j in range(1, height):
            for k in range(width):
                if k == 0:
                    left_value = M[j-1][k]
                    right_value = M[j-1][k+1]
                elif k == width - 1:
                    left_value = M[j-1][k-1]
                    right_value = M[j-1][k]
                else:
                    left_value = M[j-1][k-1]
                    right_value = M[j-1][k+1]
                center_value = M[j-1][k]
                M[j][k] += min(left_value, center_value, right_value)
        
        # 找到最小能量路径
        indices = []
        j = height - 1
        k = np.argmin(M[height-1])
        indices.append((j, k))
        while j > 0:
            if k == 0:
                idx = np.argmin([M[j-1][k], M[j-1][k+1]])
                if idx == 0:
                    k = k
                else:
                    k = k + 1
            elif k == width - 1:
                idx = np.argmin([M[j-1][k-1], M[j-1][k]])
                if idx == 0:
                    k = k - 1
                else:
                    k = k
            else:
                idx = np.argmin([M[j-1][k-1], M[j-1][k], M[j-1][k+1]])
                if idx == 0:
                    k = k - 1
                elif idx == 1:
                    k = k
                else:
                    k = k + 1            
            j -= 1
            indices.append((j, k))
        
        # 删除最小能量路径上的像素
        indices.reverse()
        for index in indices:
            img[index[0], index[1]:-1] = img[index[0], index[1]+1:]
            energy_map[index[0], index[1]:-1] = energy_map[index[0], index[1]+1:]
    
    return img[:, :new_width, :]
def seam_carving (img,scale_0,scale_1):
    img_l = seam_carve(img,scale_0)#水平方向上进行调整
    img_r_s = cv2.rotate(img_l,cv2.ROTATE_90_CLOCKWISE)
    resized_image_r = seam_carve(img_r_s,scale_1)#垂直方向调整
    resized_image = cv2.rotate(resized_image_r,cv2.ROTATE_90_COUNTERCLOCKWISE)
    return resized_image

In [ ]:

image = cv2.imread('D:/test_pic/2.jpg')

resized_image = seam_carving(image,0.99,0.95)
# 显示图像
cv2.imshow('Seam Carved Image', resized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("D:/test_pic/resized_image.jpg",resized_image)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(resized_image)
plt.show